In [1]:
## Import the required libraries and read the data into memory
## Rename the column names from spaces to underscores

import duckdb
import pandas as pd
from plotnine import (
    ggplot,
    aes,
    geom_bar,
    theme,
    element_text
)

df = pd.read_csv('./data/Trade_DetailedTradeMatrix.csv', encoding='latin-1')

colnames = []
for i in df.columns:
    i = i.replace(" ", "_")
    colnames.append(i)

df.columns = colnames

In [36]:
## This function calculates the trade dependency of a country by year by total import value.
## It fetches the top 20 countries by import value that the selected country imports from.
## For example, if you use trade_dependency('United Arab Emirates', 2021)
## You will get the top 20 countries by import value (USD $) that the U.A.E imports from
## In the year 2021. 

def trade_dependency(country, year):
        
    dataframe = duckdb.sql('''
    select Partner_Countries,
    sum(Value) as import_value,
    import_value /
        (select
            sum(Value)
            from df
            where Year = {year}
            and Element = 'Import Value'
            and Reporter_Countries = '{country}'
        ) as imp_prop
    from df
    where Year = {year}
    and Element = 'Import Value'
    and Reporter_Countries = '{country}'
    group by Partner_Countries
    having imp_prop > 0
    order by imp_prop desc;
    '''.format(year = year, country = country)).fetchdf().head(21)

    return dataframe

trade_dependency('United Arab Emirates', 2019)

,Partner_Countries,import_value,imp_prop
0,India,1908241.0,0.107462
1,Brazil,1621177.0,0.091296
2,United States of America,1482410.0,0.083482
3,Saudi Arabia,850430.0,0.047892
4,Australia,608745.0,0.034281
5,Canada,530866.0,0.029896
6,France,523323.0,0.029471
7,"China, mainland",522875.0,0.029446
8,Netherlands,512081.0,0.028838
9,New Zealand,500850.0,0.028205


In [42]:
## This function finds the trade dependants by value (USD $ of a country).
## For example, using trade_dependants('United Arab Emirates', 2021)
## Finds the top 20 countries that depend on the U.A.E
## The dependency for country A on country B is quantified by taking
## The proportion of imports from country B to country A over the total
## Imports to country A:
##
## dep_ratio = (Import value from B to A) / (Total import value to country A)


def trade_dependants(country, year):

    top_imports = duckdb.sql('''
    select Reporter_Countries,
        sum(Value) as imports
        from df
        where Year = {year}
        and Element = 'Import Value'
        and Partner_Countries = '{country}'
        group by Reporter_Countries
        having imports > 0
        order by imports desc;
        '''.format(year = year, country = country)).fetchdf().head(21)

    total_imports = duckdb.sql('''
    select Reporter_Countries,
        sum(Value) as import_value
        from df
        where Year = {year}
        and Element = 'Import Value'
        group by Reporter_Countries
        having import_value > 0
        order by import_value desc;
        '''.format(year = year)).fetchdf()

    dependants = duckdb.sql(
        '''
        drop table if exists imps;
        drop table if exists dependants;

        create table imps as
            select * from top_imports
            left join total_imports
            on top_imports.Reporter_Countries = total_imports.Reporter_Countries
            order by import_value desc;

            select Reporter_Countries, imports,
                imports / import_value as dep_ratio


                from imps
                order by dep_ratio desc;
        ''').fetchdf()
    
    return dependants

trade_dependants('United Arab Emirates', 2021)

,Reporter_Countries,imports,dep_ratio
0,Oman,1864287.0,0.350573
1,Maldives,118694.0,0.203470
2,Afghanistan,420899.0,0.186384
3,Bahrain,307722.0,0.175980
4,Syrian Arab Republic,407854.0,0.173320
5,Kuwait,791253.0,0.155162
6,Libya,640570.0,0.144555
7,Iran (Islamic Republic of),998222.0,0.107553
8,Yemen,534845.0,0.101324
9,Benin,158826.0,0.091926
